In [2]:
import pandas as pd
import numpy as np
%store -r data2
%store -r label2

df = pd.DataFrame()
df['head'] = np.array(data2)[:, 0]
df['tail'] = np.array(data2)[:, 1]
df['label'] = np.array(label2)

df.to_csv("dada_samples.csv", index=False)

In [1]:
import copy
import pickle
import pandas as pd
import numpy as np
import networkx as nx

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
from sklearn import metrics

from feature import extract_node_feature, extract_edge_feature

In [3]:
# parameters
classify_epoch = 20
classify_lr = 0.001
thres = 0.5
paras = [classify_epoch, classify_lr]

## Read data

In [4]:
%store -r G
%store -r unG
%store -r GG
%store -r G_train
%store -r unG_train
%store -r G_val
%store -r G_test_seen
%store -r train_x
%store -r train_y
%store -r val_x
%store -r val_y
%store -r test_seen_x
%store -r test_seen_y
%store -r test_x

In [5]:
with open('vgae_emb.pkl', 'rb') as f:
    emb = pickle.load(f)
with open('trans_dict.pkl', 'rb') as f:
    trans_dict = pickle.load(f)
with open('emb_16.pkl', 'rb') as f:
    emb_16 = pickle.load(f)
with open('emb_36.pkl', 'rb') as f:
    emb_36 = pickle.load(f)
with open('emb_36_feat.pkl', 'rb') as f:
    emb_36_feat = pickle.load(f)

## extract feature

In [6]:
%store -r all_f2
%store -r test_f2
%store -r test_x2

no stored variable test_x2


In [7]:
%store -r data2
%store -r label2

In [8]:
data = np.array(data2).astype('float32')
label = np.array(label2).astype('float32')
all_f = all_f2
test_f = test_f2

In [9]:
def sigmoid(x):
        return 1 / (1 + np.exp(-x))
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
new = []
for i, f in zip(data, all_f):
    head, tail = trans_dict[i[0]], trans_dict[i[1]]
    #cos = cosine_similarity(emb[head].reshape(1, -1), emb[tail].reshape(1, -1)).reshape(1)[0]
    #sig = sigmoid(np.dot(emb[head], emb[tail].T))
    new.append(np.hstack([emb[head], emb[tail], emb_16[head], emb_16[tail], emb_36[head], emb_36[tail], emb_36_feat[head], emb_36_feat[tail]]))

new = np.array(new).astype('float32')
new.shape

(568095, 208)

In [11]:
all_f = np.hstack([all_f, new])
all_f.shape

(568095, 495)

In [12]:
from sklearn.model_selection import train_test_split
split = int(len(data)*0.99)
train_f, val_f, train_y, val_y = all_f[:split], all_f[split:], label[:split], label[split:]

In [13]:
tmp = []
for i, f in zip(test_x, test_f):
    if i[0] not in GG or i[1] not in GG:
        tmp.append([0]*len(tmp[0]))
    else:
        head, tail = trans_dict[i[0]], trans_dict[i[1]]
        #cos = cosine_similarity(emb[head].reshape(1, -1), emb[tail].reshape(1, -1)).reshape(1)[0]
        #sig = sigmoid(np.dot(emb[head], emb[tail].T))
        feat = np.hstack(np.hstack([emb[head], emb[tail], emb_16[head], emb_16[tail], emb_36[head], emb_36[tail], emb_36_feat[head], emb_36_feat[tail]]))
        
        tmp.append(feat)


tmp = np.array(tmp).astype('float32')
test_f = np.hstack([test_f, tmp])
test_f.shape

(88074, 495)

## Training

In [14]:
#%store -r test_f
train_loader2 = DataLoader(
        dataset=TensorDataset(
            #torch.from_numpy(train_x[:, 0]),
            #torch.from_numpy(train_x[:, 1]),
            torch.from_numpy(train_y),
            torch.from_numpy(train_f)
        ),
        batch_size=512,
        shuffle=True,
        num_workers=8)


val_loader2 = DataLoader(
        dataset=TensorDataset(
            #torch.from_numpy(val_x[:, 0]),
            #torch.from_numpy(val_x[:, 1]),
            torch.from_numpy(val_y),
            torch.from_numpy(val_f)
        ),
        batch_size=512,
        shuffle=True,
        num_workers=8)

# validation data
#x1 = torch.from_numpy(val_x[:, 0]).cuda()
#x2 = torch.from_numpy(val_x[:, 1]).cuda()
y = torch.from_numpy(val_y).cuda()
f = torch.from_numpy(val_f).cuda()

# test_seen
#x1_test_ = torch.from_numpy(test_seen_x[:, 0]).cuda()
#x2_test_ = torch.from_numpy(test_seen_x[:, 1]).cuda()
#feat_test_ = torch.from_numpy(test_seen_f).cuda()

# test
test1 = torch.from_numpy(test_x[:, 0]).cuda()
test2 = torch.from_numpy(test_x[:, 0]).cuda()
test_f = torch.from_numpy(test_f).cuda()

In [15]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(f.size()[1], 32)
        self.fc3 = nn.Linear(32, 32)
        self.fc4 = nn.Linear(32, 32)
        self.fc2 = nn.Linear(32, 1)
        
    def forward(self, feat):
        out = F.relu(self.fc1(feat))
        out = F.relu(self.fc3(out))
        out = F.relu(self.fc4(out))
        out = F.sigmoid(self.fc2(out))
        return out

In [27]:
classifier = Classifier().cuda()
optimizer2 = torch.optim.Adam(classifier.parameters(), lr=classify_lr/10, weight_decay=0.01, amsgrad=True)

for epoch in range(50):
    print('Epoch:', epoch)
    
    total_loss = []
    cnt = 0
    correct = 0
    for e , (label, feat) in enumerate(train_loader2):
        label, feat = label.cuda(), feat.cuda()
        output = classifier(feat).squeeze()
        loss = F.binary_cross_entropy(output, label, size_average=False)
        total_loss.append(loss.item())
        
        optimizer2.zero_grad()
        loss.backward()
        optimizer2.step()
    
        cnt += output.size()[0]
        output[output>0.5] = 1
        output[output<0.5] = 0
        correct += (output.squeeze() == label.squeeze()).sum().item()
    
    # validation
    val_loss = []
    for e , (label, feat) in enumerate(val_loader2):
        label, feat = label.cuda(), feat.cuda()
        output = classifier(feat).squeeze()
        loss = F.binary_cross_entropy(output, label, size_average=False)
        val_loss.append(loss.item())
    
    print('train_loss:{:.4f}, train_acc:{:.4f}'.format(np.mean(total_loss), correct/cnt))
    
    pred = classifier(f)
    pred[pred >= 0.5] = 1
    pred[pred < 0.5] = 0
    print('val_loss:{:.4f}, val_acc:{:.4f}'.format(np.mean(val_loss), metrics.accuracy_score(val_y, pred.data.cpu().numpy())))
    print('auc', metrics.roc_auc_score(val_y, pred.data.cpu().numpy()))
    print('val one', pred.data.cpu().numpy().sum() / len(pred))

#     pred = classifier(feat_test_)
#     p = pred.detach().cpu().numpy().flatten()
#     thres = 0.5
#     p[p > thres] = 1
#     p[p < thres] = 0
#     print('test-seen acc:', np.round(p.sum()/len(p), 3))
    
    pred = classifier(test_f)
    p = pred.detach().cpu().numpy().flatten()
    thres = 0.5
    p[p > thres] = 1
    p[p < thres] = 0
    print('test one', pred.data.cpu().numpy().sum() / len(pred))
    print()
    
    
    torch.save({'state_dict': classifier.state_dict()}, 'classifier2.pt')

Epoch: 0
train_loss:276.6711, train_acc:0.7509
val_loss:192.3457, val_acc:0.8593
auc 0.8594906426037935
val one 0.465833480021123
test one 0.615729076188773

Epoch: 1
train_loss:179.1683, train_acc:0.8669
val_loss:163.9795, val_acc:0.8782
auc 0.8782820628131373
val one 0.488575954937511
test one 0.6656661319600563

Epoch: 2
train_loss:162.9855, train_acc:0.8773
val_loss:162.5369, val_acc:0.8692
auc 0.8696306998319068
val one 0.44435838760781554
test one 0.6035707100705089

Epoch: 3
train_loss:156.1494, train_acc:0.8816
val_loss:149.0298, val_acc:0.8861
auc 0.8861961534551739
val one 0.48435134659390955
test one 0.66120379772975

Epoch: 4
train_loss:151.8634, train_acc:0.8841
val_loss:145.2906, val_acc:0.8874
auc 0.8875548640346446
val one 0.4811828903362084
test one 0.6584106475804438

Epoch: 5
train_loss:148.0552, train_acc:0.8865
val_loss:142.1612, val_acc:0.8896
auc 0.8897443709407248
val one 0.4855131138883999
test one 0.6612924128431773

Epoch: 6
train_loss:144.7086, train_acc:0.8

In [28]:
pred = classifier(f)
p = pred.data.cpu().numpy().flatten()
p[p > 0.5] = 1
p[p < 0.5] = 0
print(p.sum()/len(p))
print(metrics.roc_auc_score(p, val_y))
print(metrics.accuracy_score(p, val_y))
print(metrics.confusion_matrix(p, val_y).ravel(), 'tn, fp, fn, tp')
print(metrics.confusion_matrix(p, val_y).ravel()/metrics.confusion_matrix(p, val_y).ravel().sum(), 'tn, fp, fn, tp')

0.5026227776799859
0.901947674633055
0.9019538813589156
[12726  1402  1383 12894] tn, fp, fn, tp
[0.44801971 0.04935751 0.04868861 0.45393417] tn, fp, fn, tp


In [30]:
pred = classifier(test_f)
p = pred.data.cpu().numpy().flatten()
p[p > 0.87] = 1
p[p < 0.87] = 0
print(p.sum()/ len(p))

0.49944364965824195


In [24]:
for e, i in enumerate(test_x):
    if i[0] not in GG or i[1] not in GG:
        p[e] = 1
print(p.sum()/ len(p))

0.4924268228989259


In [26]:
import pandas as pd
df = pd.read_csv('history.csv', header=None)
df[len(df.columns)] = p
df.to_csv('history.csv', header=None, index=False)

In [14]:
from xgboost import XGBRegressor
#model = XGBRegressor(n_estimators=600, learning_rate=0.1, max_depth=15, min_child_weight=11, gamma=0.2,
#                                  colsample_bytree=0.6, subsample=0.9, reg_alpha=0.1, reg_lambda=1e-05)
model = XGBRegressor(n_estimators=300, learning_rate =0.1, max_depth=10, nthread=8, objective='binary:logistic')

/home/dada/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [44]:
for i in range(5):
    model = XGBRegressor(n_estimators=300, learning_rate =0.1, max_depth=8+i, nthread=8, objective='binary:logistic', reg_alpha=0.1, reg_lambda=1)
    model.fit(train_f, train_y, early_stopping_rounds=5, eval_set=[(val_f, val_y)], eval_metric='rmse')
    pred = model.predict(test_f)
    p = pred.copy()
    p[p > 0.89] = 1
    p[p < 0.89] = 0
    for e, i in enumerate(test_x):
        if i[0] not in GG or i[1] not in GG:
            p[e] = 1
    df = pd.read_csv('history.csv', header=None)
    df[len(df.columns)] = p
    df.to_csv('history.csv', header=None, index=False)

[0]	validation_0-rmse:0.466884
Will train until validation_0-rmse hasn't improved in 5 rounds.
[1]	validation_0-rmse:0.43826
[2]	validation_0-rmse:0.413629
[3]	validation_0-rmse:0.392469
[4]	validation_0-rmse:0.374215
[5]	validation_0-rmse:0.35846
[6]	validation_0-rmse:0.345034
[7]	validation_0-rmse:0.333652
[8]	validation_0-rmse:0.323938
[9]	validation_0-rmse:0.315654
[10]	validation_0-rmse:0.308598
[11]	validation_0-rmse:0.302709
[12]	validation_0-rmse:0.297692
[13]	validation_0-rmse:0.293285
[14]	validation_0-rmse:0.289691
[15]	validation_0-rmse:0.286587
[16]	validation_0-rmse:0.284007
[17]	validation_0-rmse:0.281754
[18]	validation_0-rmse:0.279875
[19]	validation_0-rmse:0.278288
[20]	validation_0-rmse:0.276912
[21]	validation_0-rmse:0.275734
[22]	validation_0-rmse:0.274645
[23]	validation_0-rmse:0.273755
[24]	validation_0-rmse:0.272992
[25]	validation_0-rmse:0.272319
[26]	validation_0-rmse:0.271638
[27]	validation_0-rmse:0.271042
[28]	validation_0-rmse:0.270589
[29]	validation_0-rm

[34]	validation_0-rmse:0.266101
[35]	validation_0-rmse:0.265879
[36]	validation_0-rmse:0.265639
[37]	validation_0-rmse:0.265366
[38]	validation_0-rmse:0.265087
[39]	validation_0-rmse:0.264863
[40]	validation_0-rmse:0.264708
[41]	validation_0-rmse:0.264495
[42]	validation_0-rmse:0.264159
[43]	validation_0-rmse:0.264101
[44]	validation_0-rmse:0.263948
[45]	validation_0-rmse:0.263755
[46]	validation_0-rmse:0.263635
[47]	validation_0-rmse:0.263534
[48]	validation_0-rmse:0.26333
[49]	validation_0-rmse:0.263229
[50]	validation_0-rmse:0.26306
[51]	validation_0-rmse:0.26292
[52]	validation_0-rmse:0.262616
[53]	validation_0-rmse:0.262535
[54]	validation_0-rmse:0.262277
[55]	validation_0-rmse:0.262153
[56]	validation_0-rmse:0.261969
[57]	validation_0-rmse:0.261845
[58]	validation_0-rmse:0.261699
[59]	validation_0-rmse:0.261517
[60]	validation_0-rmse:0.261383
[61]	validation_0-rmse:0.26121
[62]	validation_0-rmse:0.261083
[63]	validation_0-rmse:0.26098
[64]	validation_0-rmse:0.260951
[65]	validati

[62]	validation_0-rmse:0.260457
[63]	validation_0-rmse:0.260311
[64]	validation_0-rmse:0.260199
[65]	validation_0-rmse:0.260045
[66]	validation_0-rmse:0.259928
[67]	validation_0-rmse:0.2598
[68]	validation_0-rmse:0.259616
[69]	validation_0-rmse:0.259616
[70]	validation_0-rmse:0.259546
[71]	validation_0-rmse:0.259478
[72]	validation_0-rmse:0.259399
[73]	validation_0-rmse:0.259373
[74]	validation_0-rmse:0.259295
[75]	validation_0-rmse:0.259292
[76]	validation_0-rmse:0.259159
[77]	validation_0-rmse:0.259169
[78]	validation_0-rmse:0.259114
[79]	validation_0-rmse:0.258998
[80]	validation_0-rmse:0.258868
[81]	validation_0-rmse:0.258796
[82]	validation_0-rmse:0.258764
[83]	validation_0-rmse:0.258638
[84]	validation_0-rmse:0.258477
[85]	validation_0-rmse:0.258396
[86]	validation_0-rmse:0.258389
[87]	validation_0-rmse:0.258405
[88]	validation_0-rmse:0.258228
[89]	validation_0-rmse:0.258166
[90]	validation_0-rmse:0.258178
[91]	validation_0-rmse:0.258113
[92]	validation_0-rmse:0.258007
[93]	valid

[14]	validation_0-rmse:0.283687
[15]	validation_0-rmse:0.28073
[16]	validation_0-rmse:0.278103
[17]	validation_0-rmse:0.27597
[18]	validation_0-rmse:0.274001
[19]	validation_0-rmse:0.272286
[20]	validation_0-rmse:0.271003
[21]	validation_0-rmse:0.269846
[22]	validation_0-rmse:0.268921
[23]	validation_0-rmse:0.267929
[24]	validation_0-rmse:0.267225
[25]	validation_0-rmse:0.266655
[26]	validation_0-rmse:0.266129
[27]	validation_0-rmse:0.265626
[28]	validation_0-rmse:0.265106
[29]	validation_0-rmse:0.264707
[30]	validation_0-rmse:0.264239
[31]	validation_0-rmse:0.263876
[32]	validation_0-rmse:0.263539
[33]	validation_0-rmse:0.263322
[34]	validation_0-rmse:0.263064
[35]	validation_0-rmse:0.26281
[36]	validation_0-rmse:0.26257
[37]	validation_0-rmse:0.262399
[38]	validation_0-rmse:0.262211
[39]	validation_0-rmse:0.261969
[40]	validation_0-rmse:0.26172
[41]	validation_0-rmse:0.261684
[42]	validation_0-rmse:0.261603
[43]	validation_0-rmse:0.261464
[44]	validation_0-rmse:0.261303
[45]	validati

In [45]:
for i in range(5):
    model = XGBRegressor(n_estimators=200+i*50, learning_rate =0.1, max_depth=10, nthread=8, objective='binary:logistic', reg_alpha=0.1, reg_lambda=1)
    model.fit(train_f, train_y, early_stopping_rounds=5, eval_set=[(val_f, val_y)], eval_metric='rmse')
    pred = model.predict(test_f)
    p = pred.copy()
    p[p > 0.89] = 1
    p[p < 0.89] = 0
    for e, i in enumerate(test_x):
        if i[0] not in GG or i[1] not in GG:
            p[e] = 1
    df = pd.read_csv('history.csv', header=None)
    df[len(df.columns)] = p
    df.to_csv('history.csv', header=None, index=False)

[0]	validation_0-rmse:0.466316
Will train until validation_0-rmse hasn't improved in 5 rounds.
[1]	validation_0-rmse:0.437215
[2]	validation_0-rmse:0.412118
[3]	validation_0-rmse:0.390479
[4]	validation_0-rmse:0.371762
[5]	validation_0-rmse:0.35579
[6]	validation_0-rmse:0.34217
[7]	validation_0-rmse:0.330542
[8]	validation_0-rmse:0.320644
[9]	validation_0-rmse:0.312279
[10]	validation_0-rmse:0.305349
[11]	validation_0-rmse:0.299251
[12]	validation_0-rmse:0.294246
[13]	validation_0-rmse:0.290046
[14]	validation_0-rmse:0.286403
[15]	validation_0-rmse:0.283341
[16]	validation_0-rmse:0.280734
[17]	validation_0-rmse:0.278455
[18]	validation_0-rmse:0.276566
[19]	validation_0-rmse:0.27499
[20]	validation_0-rmse:0.273489
[21]	validation_0-rmse:0.27232
[22]	validation_0-rmse:0.271231
[23]	validation_0-rmse:0.270305
[24]	validation_0-rmse:0.269503
[25]	validation_0-rmse:0.268847
[26]	validation_0-rmse:0.268124
[27]	validation_0-rmse:0.267567
[28]	validation_0-rmse:0.267082
[29]	validation_0-rmse

[89]	validation_0-rmse:0.258166
[90]	validation_0-rmse:0.258178
[91]	validation_0-rmse:0.258113
[92]	validation_0-rmse:0.258007
[93]	validation_0-rmse:0.257946
[94]	validation_0-rmse:0.257878
[95]	validation_0-rmse:0.257798
[96]	validation_0-rmse:0.257781
[97]	validation_0-rmse:0.257725
[98]	validation_0-rmse:0.257675
[99]	validation_0-rmse:0.257568
[100]	validation_0-rmse:0.257496
[101]	validation_0-rmse:0.257449
[102]	validation_0-rmse:0.257451
[103]	validation_0-rmse:0.257364
[104]	validation_0-rmse:0.257378
[105]	validation_0-rmse:0.25732
[106]	validation_0-rmse:0.257292
[107]	validation_0-rmse:0.257302
[108]	validation_0-rmse:0.257271
[109]	validation_0-rmse:0.25723
[110]	validation_0-rmse:0.257173
[111]	validation_0-rmse:0.257161
[112]	validation_0-rmse:0.257076
[113]	validation_0-rmse:0.257079
[114]	validation_0-rmse:0.257071
[115]	validation_0-rmse:0.257038
[116]	validation_0-rmse:0.25703
[117]	validation_0-rmse:0.257021
[118]	validation_0-rmse:0.256934
[119]	validation_0-rmse:

[13]	validation_0-rmse:0.290046
[14]	validation_0-rmse:0.286403
[15]	validation_0-rmse:0.283341
[16]	validation_0-rmse:0.280734
[17]	validation_0-rmse:0.278455
[18]	validation_0-rmse:0.276566
[19]	validation_0-rmse:0.27499
[20]	validation_0-rmse:0.273489
[21]	validation_0-rmse:0.27232
[22]	validation_0-rmse:0.271231
[23]	validation_0-rmse:0.270305
[24]	validation_0-rmse:0.269503
[25]	validation_0-rmse:0.268847
[26]	validation_0-rmse:0.268124
[27]	validation_0-rmse:0.267567
[28]	validation_0-rmse:0.267082
[29]	validation_0-rmse:0.266516
[30]	validation_0-rmse:0.266166
[31]	validation_0-rmse:0.265853
[32]	validation_0-rmse:0.265567
[33]	validation_0-rmse:0.265342
[34]	validation_0-rmse:0.265055
[35]	validation_0-rmse:0.264782
[36]	validation_0-rmse:0.264557
[37]	validation_0-rmse:0.264408
[38]	validation_0-rmse:0.264222
[39]	validation_0-rmse:0.264035
[40]	validation_0-rmse:0.263789
[41]	validation_0-rmse:0.263579
[42]	validation_0-rmse:0.263473
[43]	validation_0-rmse:0.263146
[44]	valid

KeyboardInterrupt: 

In [47]:
for i in range(3):
    model = XGBRegressor(n_estimators=300, learning_rate =0.1, max_depth=5+i, nthread=8, objective='binary:logistic', reg_alpha=0.1, reg_lambda=1)
    model.fit(train_f, train_y, early_stopping_rounds=5, eval_set=[(val_f, val_y)], eval_metric='rmse')
    pred = model.predict(test_f)
    p = pred.copy()
    p[p > 0.89] = 1
    p[p < 0.89] = 0
    for e, i in enumerate(test_x):
        if i[0] not in GG or i[1] not in GG:
            p[e] = 1
    df = pd.read_csv('history.csv', header=None)
    df[len(df.columns)] = p
    df.to_csv('history.csv', header=None, index=False)

[0]	validation_0-rmse:0.468282
Will train until validation_0-rmse hasn't improved in 5 rounds.
[1]	validation_0-rmse:0.441025
[2]	validation_0-rmse:0.417442
[3]	validation_0-rmse:0.39727
[4]	validation_0-rmse:0.379901
[5]	validation_0-rmse:0.365023
[6]	validation_0-rmse:0.352424
[7]	validation_0-rmse:0.341654
[8]	validation_0-rmse:0.332426
[9]	validation_0-rmse:0.32462
[10]	validation_0-rmse:0.318048
[11]	validation_0-rmse:0.312429
[12]	validation_0-rmse:0.307599
[13]	validation_0-rmse:0.303657
[14]	validation_0-rmse:0.300287
[15]	validation_0-rmse:0.29746
[16]	validation_0-rmse:0.294933
[17]	validation_0-rmse:0.292849
[18]	validation_0-rmse:0.291069
[19]	validation_0-rmse:0.289564
[20]	validation_0-rmse:0.288221
[21]	validation_0-rmse:0.287054
[22]	validation_0-rmse:0.285986
[23]	validation_0-rmse:0.285102
[24]	validation_0-rmse:0.284373
[25]	validation_0-rmse:0.283572
[26]	validation_0-rmse:0.282932
[27]	validation_0-rmse:0.282174
[28]	validation_0-rmse:0.281567
[29]	validation_0-rms

[251]	validation_0-rmse:0.26023
[252]	validation_0-rmse:0.260178
[253]	validation_0-rmse:0.260142
[254]	validation_0-rmse:0.260089
[255]	validation_0-rmse:0.260071
[256]	validation_0-rmse:0.260046
[257]	validation_0-rmse:0.260013
[258]	validation_0-rmse:0.259978
[259]	validation_0-rmse:0.259935
[260]	validation_0-rmse:0.259916
[261]	validation_0-rmse:0.259905
[262]	validation_0-rmse:0.259884
[263]	validation_0-rmse:0.259847
[264]	validation_0-rmse:0.259824
[265]	validation_0-rmse:0.259837
[266]	validation_0-rmse:0.259801
[267]	validation_0-rmse:0.259798
[268]	validation_0-rmse:0.259801
[269]	validation_0-rmse:0.259767
[270]	validation_0-rmse:0.259738
[271]	validation_0-rmse:0.259718
[272]	validation_0-rmse:0.259687
[273]	validation_0-rmse:0.259663
[274]	validation_0-rmse:0.259651
[275]	validation_0-rmse:0.259686
[276]	validation_0-rmse:0.259701
[277]	validation_0-rmse:0.2597
[278]	validation_0-rmse:0.259711
[279]	validation_0-rmse:0.259665
Stopping. Best iteration:
[274]	validation_0-r

[220]	validation_0-rmse:0.257839
[221]	validation_0-rmse:0.257841
[222]	validation_0-rmse:0.257812
[223]	validation_0-rmse:0.257788
[224]	validation_0-rmse:0.257754
[225]	validation_0-rmse:0.257729
[226]	validation_0-rmse:0.257726
[227]	validation_0-rmse:0.257704
[228]	validation_0-rmse:0.257641
[229]	validation_0-rmse:0.257646
[230]	validation_0-rmse:0.257614
[231]	validation_0-rmse:0.257578
[232]	validation_0-rmse:0.25757
[233]	validation_0-rmse:0.257573
[234]	validation_0-rmse:0.257547
[235]	validation_0-rmse:0.257529
[236]	validation_0-rmse:0.257529
[237]	validation_0-rmse:0.257549
[238]	validation_0-rmse:0.257521
[239]	validation_0-rmse:0.257516
[240]	validation_0-rmse:0.257493
[241]	validation_0-rmse:0.257464
[242]	validation_0-rmse:0.257401
[243]	validation_0-rmse:0.257405
[244]	validation_0-rmse:0.257347
[245]	validation_0-rmse:0.257345
[246]	validation_0-rmse:0.257352
[247]	validation_0-rmse:0.257322
[248]	validation_0-rmse:0.257243
[249]	validation_0-rmse:0.25721
[250]	valida

[204]	validation_0-rmse:0.256824
[205]	validation_0-rmse:0.256815
[206]	validation_0-rmse:0.256763
[207]	validation_0-rmse:0.256738
[208]	validation_0-rmse:0.256729
[209]	validation_0-rmse:0.256685
[210]	validation_0-rmse:0.256642
[211]	validation_0-rmse:0.256633
[212]	validation_0-rmse:0.256623
[213]	validation_0-rmse:0.256564
[214]	validation_0-rmse:0.256553
[215]	validation_0-rmse:0.256535
[216]	validation_0-rmse:0.256536
[217]	validation_0-rmse:0.256512
[218]	validation_0-rmse:0.256522
[219]	validation_0-rmse:0.256504
[220]	validation_0-rmse:0.256476
[221]	validation_0-rmse:0.256467
[222]	validation_0-rmse:0.256441
[223]	validation_0-rmse:0.256436
[224]	validation_0-rmse:0.256452
[225]	validation_0-rmse:0.256446
[226]	validation_0-rmse:0.256437
[227]	validation_0-rmse:0.256424
[228]	validation_0-rmse:0.256374
[229]	validation_0-rmse:0.256321
[230]	validation_0-rmse:0.256283
[231]	validation_0-rmse:0.256266
[232]	validation_0-rmse:0.256234
[233]	validation_0-rmse:0.256247
[234]	vali

In [50]:
for i in range(5):
    model = XGBRegressor(n_estimators=300, learning_rate =0.1, max_depth=8, seed=87*(i+1), nthread=8, objective='binary:logistic', reg_alpha=0.1, reg_lambda=1)
    model.fit(train_f, train_y, early_stopping_rounds=5, eval_set=[(val_f, val_y)], eval_metric='rmse')
    pred = model.predict(test_f)
    p = pred.copy()
    p[p > 0.89] = 1
    p[p < 0.89] = 0
    for e, i in enumerate(test_x):
        if i[0] not in GG or i[1] not in GG:
            p[e] = 1
    df = pd.read_csv('history.csv', header=None)
    df[len(df.columns)] = p
    df.to_csv('history.csv', header=None, index=False)

[0]	validation_0-rmse:0.466884
Will train until validation_0-rmse hasn't improved in 5 rounds.
[1]	validation_0-rmse:0.43826
[2]	validation_0-rmse:0.413629
[3]	validation_0-rmse:0.392469
[4]	validation_0-rmse:0.374215
[5]	validation_0-rmse:0.35846
[6]	validation_0-rmse:0.345034
[7]	validation_0-rmse:0.333652
[8]	validation_0-rmse:0.323938
[9]	validation_0-rmse:0.315654
[10]	validation_0-rmse:0.308598
[11]	validation_0-rmse:0.302709
[12]	validation_0-rmse:0.297692
[13]	validation_0-rmse:0.293285
[14]	validation_0-rmse:0.289691
[15]	validation_0-rmse:0.286587
[16]	validation_0-rmse:0.284007
[17]	validation_0-rmse:0.281754
[18]	validation_0-rmse:0.279875
[19]	validation_0-rmse:0.278288
[20]	validation_0-rmse:0.276912
[21]	validation_0-rmse:0.275734
[22]	validation_0-rmse:0.274645
[23]	validation_0-rmse:0.273755
[24]	validation_0-rmse:0.272992
[25]	validation_0-rmse:0.272319
[26]	validation_0-rmse:0.271638
[27]	validation_0-rmse:0.271042
[28]	validation_0-rmse:0.270589
[29]	validation_0-rm

[34]	validation_0-rmse:0.268295
[35]	validation_0-rmse:0.268068
[36]	validation_0-rmse:0.267761
[37]	validation_0-rmse:0.267514
[38]	validation_0-rmse:0.267226
[39]	validation_0-rmse:0.267041
[40]	validation_0-rmse:0.266632
[41]	validation_0-rmse:0.266383
[42]	validation_0-rmse:0.266178
[43]	validation_0-rmse:0.266008
[44]	validation_0-rmse:0.265874
[45]	validation_0-rmse:0.265713
[46]	validation_0-rmse:0.265568
[47]	validation_0-rmse:0.265308
[48]	validation_0-rmse:0.265098
[49]	validation_0-rmse:0.265061
[50]	validation_0-rmse:0.264912
[51]	validation_0-rmse:0.264792
[52]	validation_0-rmse:0.264612
[53]	validation_0-rmse:0.264493
[54]	validation_0-rmse:0.264252
[55]	validation_0-rmse:0.264106
[56]	validation_0-rmse:0.263918
[57]	validation_0-rmse:0.263723
[58]	validation_0-rmse:0.263624
[59]	validation_0-rmse:0.263369
[60]	validation_0-rmse:0.263223
[61]	validation_0-rmse:0.263015
[62]	validation_0-rmse:0.262873
[63]	validation_0-rmse:0.262786
[64]	validation_0-rmse:0.262649
[65]	val

[70]	validation_0-rmse:0.26198
[71]	validation_0-rmse:0.261928
[72]	validation_0-rmse:0.261751
[73]	validation_0-rmse:0.261703
[74]	validation_0-rmse:0.261655
[75]	validation_0-rmse:0.261564
[76]	validation_0-rmse:0.261532
[77]	validation_0-rmse:0.261502
[78]	validation_0-rmse:0.261331
[79]	validation_0-rmse:0.261185
[80]	validation_0-rmse:0.261116
[81]	validation_0-rmse:0.260918
[82]	validation_0-rmse:0.260801
[83]	validation_0-rmse:0.260701
[84]	validation_0-rmse:0.260573
[85]	validation_0-rmse:0.260506
[86]	validation_0-rmse:0.260389
[87]	validation_0-rmse:0.260387
[88]	validation_0-rmse:0.260268
[89]	validation_0-rmse:0.260138
[90]	validation_0-rmse:0.260042
[91]	validation_0-rmse:0.259885
[92]	validation_0-rmse:0.259795
[93]	validation_0-rmse:0.259745
[94]	validation_0-rmse:0.259637
[95]	validation_0-rmse:0.259533
[96]	validation_0-rmse:0.259483
[97]	validation_0-rmse:0.259448
[98]	validation_0-rmse:0.259404
[99]	validation_0-rmse:0.259283
[100]	validation_0-rmse:0.259169
[101]	va

[106]	validation_0-rmse:0.25865
[107]	validation_0-rmse:0.25869
[108]	validation_0-rmse:0.258672
[109]	validation_0-rmse:0.258539
[110]	validation_0-rmse:0.258513
[111]	validation_0-rmse:0.258437
[112]	validation_0-rmse:0.258325
[113]	validation_0-rmse:0.258272
[114]	validation_0-rmse:0.258212
[115]	validation_0-rmse:0.258111
[116]	validation_0-rmse:0.25804
[117]	validation_0-rmse:0.257949
[118]	validation_0-rmse:0.257891
[119]	validation_0-rmse:0.257827
[120]	validation_0-rmse:0.257772
[121]	validation_0-rmse:0.257779
[122]	validation_0-rmse:0.257674
[123]	validation_0-rmse:0.257628
[124]	validation_0-rmse:0.257499
[125]	validation_0-rmse:0.25744
[126]	validation_0-rmse:0.257374
[127]	validation_0-rmse:0.257337
[128]	validation_0-rmse:0.257246
[129]	validation_0-rmse:0.257231
[130]	validation_0-rmse:0.257202
[131]	validation_0-rmse:0.2572
[132]	validation_0-rmse:0.257219
[133]	validation_0-rmse:0.257168
[134]	validation_0-rmse:0.257144
[135]	validation_0-rmse:0.257121
[136]	validation

[141]	validation_0-rmse:0.256906
[142]	validation_0-rmse:0.256922
[143]	validation_0-rmse:0.256863
[144]	validation_0-rmse:0.256817
[145]	validation_0-rmse:0.256795
[146]	validation_0-rmse:0.256776
[147]	validation_0-rmse:0.256745
[148]	validation_0-rmse:0.256696
[149]	validation_0-rmse:0.256669
[150]	validation_0-rmse:0.25665
[151]	validation_0-rmse:0.256612
[152]	validation_0-rmse:0.256526
[153]	validation_0-rmse:0.256514
[154]	validation_0-rmse:0.256478
[155]	validation_0-rmse:0.256399
[156]	validation_0-rmse:0.256383
[157]	validation_0-rmse:0.256347
[158]	validation_0-rmse:0.25633
[159]	validation_0-rmse:0.256304
[160]	validation_0-rmse:0.256272
[161]	validation_0-rmse:0.256227
[162]	validation_0-rmse:0.256211
[163]	validation_0-rmse:0.256212
[164]	validation_0-rmse:0.256205
[165]	validation_0-rmse:0.256152
[166]	validation_0-rmse:0.256069
[167]	validation_0-rmse:0.256025
[168]	validation_0-rmse:0.255982
[169]	validation_0-rmse:0.255916
[170]	validation_0-rmse:0.255846
[171]	valida

In [15]:
# last 0.2558
model.fit(train_f, train_y, early_stopping_rounds=5, eval_set=[(val_f, val_y)], verbose=True, eval_metric='rmse')

[0]	validation_0-rmse:0.467269
Will train until validation_0-rmse hasn't improved in 5 rounds.
[1]	validation_0-rmse:0.438936
[2]	validation_0-rmse:0.414348
[3]	validation_0-rmse:0.393281
[4]	validation_0-rmse:0.375278
[5]	validation_0-rmse:0.359746
[6]	validation_0-rmse:0.346299
[7]	validation_0-rmse:0.334987
[8]	validation_0-rmse:0.325391
[9]	validation_0-rmse:0.317647
[10]	validation_0-rmse:0.310695
[11]	validation_0-rmse:0.304851
[12]	validation_0-rmse:0.299856
[13]	validation_0-rmse:0.295382
[14]	validation_0-rmse:0.29165
[15]	validation_0-rmse:0.288743
[16]	validation_0-rmse:0.286198
[17]	validation_0-rmse:0.283983
[18]	validation_0-rmse:0.281995
[19]	validation_0-rmse:0.280286
[20]	validation_0-rmse:0.278824
[21]	validation_0-rmse:0.277843
[22]	validation_0-rmse:0.276693
[23]	validation_0-rmse:0.276041
[24]	validation_0-rmse:0.275277
[25]	validation_0-rmse:0.274751
[26]	validation_0-rmse:0.27441
[27]	validation_0-rmse:0.273866
[28]	validation_0-rmse:0.273253
[29]	validation_0-rm

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=1, missing=None, n_estimators=300, nthread=8,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [55]:
with open('xgb.pkl', 'wb') as f:
    pickle.dump(model, f)

In [18]:
pred = model.predict(val_f)
p = pred.copy()
p[p > 0.5] = 1
p[p < 0.5] = 0
print(p.sum()/len(p))
print(metrics.roc_auc_score(p, val_y))
print(metrics.accuracy_score(p, val_y))
print(metrics.confusion_matrix(p, val_y).ravel(), 'tn, fp, fn, tp')
print(metrics.confusion_matrix(p, val_y).ravel()/metrics.confusion_matrix(p, val_y).ravel().sum(), 'tn, fp, fn, tp')

0.49093469459602185
0.9042549221477183
0.9040661855307164
[2585  307  238 2551] tn, fp, fn, tp
[0.45502552 0.05403978 0.04189403 0.44904066] tn, fp, fn, tp


In [17]:
with open('xgb.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
pred = model.predict(all_f)
p = pred.copy()
p[p >= 0.55] = 1
p[p < 0.55] = 0
print(metrics.accuracy_score(p, label))

In [23]:
pred = model.predict(test_f)
p = pred.copy()
p[p > 0.89] = 1
p[p < 0.89] = 0
print(p.sum()/ len(p))

0.4910075618230125


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def plot_importance(m):
    feature_list = []
    score = []
    fscore = m.booster().get_fscore()
    for i in fscore:
        feature_list.append(int(i[1:]))
        score.append(fscore[i])
    
    df_plot = pd.DataFrame({'features': feature_list, 'importances': score})
    #df_plot = df_plot.sort_values('importances', ascending=False)
    df_plot = df_plot.sort_values('features', ascending=True)
    df_plot.features.apply(str)

    plt.figure(figsize=[20,10])
    sns.barplot(y = df_plot.importances, x = df_plot.features)
    plt.title('Importances of Features Plot')
    plt.show()
    return df_plot

In [ ]:
df_plot = plot_importance(model)

In [ ]:
df_plot.sort_values('importances', ascending=False).head(20)

In [ ]:
np.array(lll).sum() / len(lll)

In [34]:
with open('vgae_emb.pkl', 'rb') as f:
    emb = pickle.load(f)
with open('trans_dict.pkl', 'rb') as f:
    trans_dict = pickle.load(f)

In [100]:
len(train_x)

483902

In [96]:
X = []
Y = []

for i, j, f in zip(train_x, train_y, train_f):
    head, tail = trans_dict[i[0]], trans_dict[i[1]]
    feat = np.hstack([emb[head], emb[tail]])
    X.append(feat)
    Y.append(j)

In [ ]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0, tol=1e-4)
clf.fit(X, Y)

In [44]:
pred = model.predict(X)
pred[pred>=0.5] = 1
pred[pred<0.5] = 0
metrics.accuracy_score(Y, pred)

0.9782869258651545

In [95]:
val_X = []
val_Y = []

for i, j, f in zip(val_x, val_y, val_f):
    head, tail = trans_dict[i[0]], trans_dict[i[1]]
    feat = np.hstack([emb[head], emb[tail]])
    val_X.append(feat)
    val_Y.append(j)

In [50]:
pred = model.predict(val_X)
pred[pred>=0.5] = 1
pred[pred<0.5] = 0
metrics.accuracy_score(val_Y, pred)

0.9331088893423111

In [57]:
pred = np.array(pred)
p = pred.copy()
p[p>0.8] = 1
p[p<0.8] = 0
print(p.sum()/len(pred), cnt)

0.4870563389876695 383


In [94]:
new = []
for i, f in zip(test_x, test_f):
    if i[0] not in GG or i[1] not in GG:
        new.append([0]*32)
    else:
        head, tail = trans_dict[i[0]], trans_dict[i[1]]
        feat = np.hstack([emb[head], emb[tail]])
        new.append(feat)
new = np.array(new).astype('float32')

In [52]:
pred = []
cnt = 0

for i, f in zip(test_x, test_f):
    if i[0] not in GG or i[1] not in GG:
        pred.append(1)
        cnt += 1
    else:
        head, tail = trans_dict[i[0]], trans_dict[i[1]]
        feat = np.hstack([emb[head], emb[tail], f[261:]])
        tmp = model.predict([feat])[0]
        pred.append(tmp)


pred = np.array(pred)
#pred[pred>0.95] = 1
#pred[pred<0.95] = 0
#print(pred.sum()/len(pred), cnt)

In [43]:
model.fit(X, Y, early_stopping_rounds=20, eval_set=[(val_X, val_Y)], verbose=True)

[0]	validation_0-rmse:0.46366
Will train until validation_0-rmse hasn't improved in 20 rounds.
[1]	validation_0-rmse:0.431123
[2]	validation_0-rmse:0.403005
[3]	validation_0-rmse:0.3779
[4]	validation_0-rmse:0.356275
[5]	validation_0-rmse:0.338167
[6]	validation_0-rmse:0.322806
[7]	validation_0-rmse:0.309111
[8]	validation_0-rmse:0.297524
[9]	validation_0-rmse:0.287207
[10]	validation_0-rmse:0.27857
[11]	validation_0-rmse:0.271831
[12]	validation_0-rmse:0.265342
[13]	validation_0-rmse:0.259807
[14]	validation_0-rmse:0.255551
[15]	validation_0-rmse:0.251922
[16]	validation_0-rmse:0.249023
[17]	validation_0-rmse:0.24634
[18]	validation_0-rmse:0.244302
[19]	validation_0-rmse:0.242481
[20]	validation_0-rmse:0.240966
[21]	validation_0-rmse:0.239857
[22]	validation_0-rmse:0.238736
[23]	validation_0-rmse:0.237756
[24]	validation_0-rmse:0.236878
[25]	validation_0-rmse:0.236139
[26]	validation_0-rmse:0.235461
[27]	validation_0-rmse:0.234983
[28]	validation_0-rmse:0.234378
[29]	validation_0-rmse

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.6,
       gamma=0.2, learning_rate=0.1, max_delta_step=0, max_depth=15,
       min_child_weight=11, missing=None, n_estimators=1000, nthread=-1,
       objective='reg:linear', reg_alpha=0.1, reg_lambda=1e-05,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.9)

In [28]:
with open('pred.txt', 'w') as f:
    for i in p.astype(int):
        print(i, file=f)
!python3 pred-txt-to-csv.py pred.txt

In [27]:
# ensemble
import pandas as pd
df = pd.read_csv('history.csv', header=None)
print(len(df.columns), 'columns')
ens_pred = np.mean(df.values[:, 15:], 1)
print('shape', ens_pred.shape)
ens_pred[ens_pred>=0.55] = 1
ens_pred[ens_pred<0.55] = 0
print('sum', ens_pred.sum()/len(ens_pred))


for e, i in enumerate(test_x):
    if i[0] not in GG or i[1] not in GG:
        ens_pred[e] = 1
        
print('sum', ens_pred.sum()/len(ens_pred))
p = ens_pred.copy()

39 columns
shape (88074,)
sum 0.49177963984830936
sum 0.49177963984830936
